# ⚛️ H₂O — Bond Angle Optimization via Noiseless VQE (UCCSD)

This notebook scans **H–O–H bond angles** and computes the **ground-state energy**
of the water molecule using a **noiseless VQE** simulation with a **UCCSD** ansatz.

For each angle we:
- Define the H₂O geometry (fixed OH bond length, variable angle)  
- Rebuild the Hamiltonian in the STO-3G basis  
- Run noiseless VQE with singles+doubles excitations  
- Average results over multiple seeds  
- Plot **energy vs bond angle** with error bars  

Goal: approximate the **equilibrium bond angle** of H₂O.

In [ ]:
import pennylane as qml
from pennylane import numpy as np
from pennylane import qchem
import matplotlib.pyplot as plt

import sys, os, json
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

from vqe.vqe_utils import (
    IMG_DIR,
    excitation_ansatz,
    get_optimizer,
    set_seed,
    make_run_config_dict,
    run_signature,
    find_existing_run,
    save_run_record,
    ensure_dirs,
    build_run_filename,
)

ensure_dirs()

# 🧬 Part 1 — Global Configuration

- Molecule: H₂O  
- Basis: STO-3G  
- Electrons: 10  
- Ansatz: UCCSD  
- Optimizer: Adam  
- Multiple seeds for statistical averaging  
- Bond angle sweep: 100° → 109°  

In [ ]:
symbols = ["O", "H", "H"]
basis = "STO-3G"
electrons = 10

ansatz_desc = "UCCSD"
optimizer_name = "Adam"
stepsize = 0.2
max_iterations = 20

seeds = np.arange(0, 3)
angles = np.linspace(100, 109, 5)   # degrees

# 📐 Part 2 — Parametrized H₂O Geometry  

We fix the O–H bond length and vary only the **H–O–H bond angle**.

In [ ]:
def water_geometry(angle_deg, bond_length=0.9584):
    """Return H₂O coordinates for a given bond angle (degrees)."""
    angle_rad = np.deg2rad(angle_deg)
    x = bond_length * np.sin(angle_rad / 2)
    z = bond_length * np.cos(angle_rad / 2)
    return np.array([
        [0.0, 0.0, 0.0],   # O
        [x,   0.0, z],     # H1
        [-x,  0.0, z],     # H2
    ])

# ⚙️ Part 3 — Geometry-Specific UCCSD VQE QNode

Each angle yields:
- A new Hamiltonian  
- A new HF reference state  
- A new excitation structure  

From these we build a local UCCSD QNode.

In [ ]:
def make_vqe_qnode(hamiltonian, hf_state, singles, doubles, num_wires, return_state=False):
    """Create a noiseless UCCSD VQE QNode for one geometry."""
    dev = qml.device("default.qubit", wires=num_wires)

    @qml.qnode(dev, diff_method="parameter-shift")
    def circuit(params):
        excitation_ansatz(
            params,
            wires=range(num_wires),
            hf_state=hf_state,
            excitations=(singles, doubles),
            excitation_type="both",
        )
        return qml.state() if return_state else qml.expval(hamiltonian)

    return circuit

# 💾 Part 4 — Cache-Aware Single-Angle Runner

Each configuration is hashed, so identical runs are automatically reused.

In [ ]:
def run_vqe_for_angle(angle_deg, seed):
    """Run noiseless UCCSD VQE for H₂O at a given angle + seed."""
    set_seed(seed)

    # --- Geometry + electronic structure ---
    coordinates = water_geometry(angle_deg)
    hamiltonian, qubits = qchem.molecular_hamiltonian(
        symbols, coordinates, charge=0, basis=basis
    )
    hf = qchem.hf_state(electrons, qubits)
    singles, doubles = qchem.excitations(electrons, qubits)

    # --- Build unique configuration signature ---
    cfg = make_run_config_dict(
        symbols=symbols,
        coordinates=coordinates,
        basis=basis,
        ansatz_desc=ansatz_desc,
        optimizer_name=optimizer_name,
        stepsize=stepsize,
        max_iterations=max_iterations,
        seed=seed,
        noisy=False,
    )

    sig = run_signature(cfg)
    fname = build_run_filename(
        f"H2O_BondAngle_{angle_deg:.1f}deg", optimizer_name, seed, sig
    )
    existing = find_existing_run(sig)

    # --- Load cached result ---
    if existing:
        with open(existing) as f:
            rec = json.load(f)
        print(f"[reuse] angle {angle_deg:.1f}° → {existing}")
        energies = rec["energies"]
        return float(energies[-1]), energies

    # --- Fresh VQE run ---
    cost_fn = make_vqe_qnode(
        hamiltonian, hf, singles, doubles, qubits, return_state=False
    )
    opt = get_optimizer(optimizer_name, stepsize)

    params = np.zeros(len(singles) + len(doubles))
    energies = [cost_fn(params)]

    for _ in range(max_iterations):
        params, e_prev = opt.step_and_cost(cost_fn, params)
        energies.append(e_prev)

    energies[-1] = float(cost_fn(params))
    final_energy = float(energies[-1])

    # --- Save run ---
    rec = {
        "config": cfg,
        "energies": np.asarray(energies).tolist(),
        "final_params": np.asarray(params).tolist(),
        "metadata": {"angle_deg": float(angle_deg)},
    }

    persisted = save_run_record(fname, rec)
    print(f"[saved] {optimizer_name}: {fname}")
    print(f"[mirrored] {persisted}")

    return final_energy, energies

# 🚀 Part 5 — Full Bond-Angle Sweep  

For each angle we run several seeds and compute:
- mean energy  
- standard deviation  

In [ ]:
energy_means = []
energy_stds = []

for angle in angles:
    print(f"\nRunning VQE for angle {angle:.1f}°")
    results = []

    for seed in seeds:
        final_E, _ = run_vqe_for_angle(angle, seed)
        results.append(final_E)

    energy_means.append(np.mean(results))
    energy_stds.append(np.std(results))

energy_means = np.array(energy_means)
energy_stds = np.array(energy_stds)

# 📉 Part 6 — Energy vs Bond Angle Plot

In [ ]:
plt.figure(figsize=(8, 5))
plt.errorbar(angles, energy_means, yerr=energy_stds, fmt="o-", capsize=4)

plt.xlabel("H–O–H Angle (°)")
plt.ylabel("Ground-state energy (Ha)")
plt.title("H₂O VQE: Energy vs Bond Angle (Noiseless, UCCSD)")
plt.grid(True)
plt.tight_layout()

plt.savefig(f"{IMG_DIR}/H2O_Bond_Angle_Scan.png", dpi=300)
plt.show()

# 🏆 Part 7 — Optimal Bond Angle

In [ ]:
idx = int(np.argmin(energy_means))
opt_angle = angles[idx]
opt_energy = energy_means[idx]

print(f"Optimal bond angle: {opt_angle:.2f}°")
print(f"Minimum mean energy: {opt_energy:.8f} ± {energy_stds[idx]:.8f} Ha")